In [1]:
from PIL import Image

import tensorflow as tf
from random import uniform

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
#%matplotlib inline

import glob
import scipy
import scipy.misc
from scipy import ndimage
from scipy.misc import toimage
from enum import Enum
from sklearn.utils import shuffle
import ntpath

np.set_printoptions(threshold=np.inf)

In [2]:
WIDTH = 32
HEIGHT = 32
DEPTH = 3

In [3]:
def sharpen_image(image):
    #blurred_image = ndimage.gaussian_filter(image, 2)
    #filter_blurred = ndimage.gaussian_filter(blurred_image, 1)
    #alpha =0 #30
    #sharpened_image = (blurred_image + alpha) * (blurred_image - filter_blurred)
    threshold = 25
    sharpened_image = image
    sharpened_image[sharpened_image > threshold] = 250
    sharpened_image[sharpened_image <= threshold] = 0
    return sharpened_image

In [4]:
def gradient_of_image(image):
    
    # This filten makes edges wider 
    image = ndimage.gaussian_filter(image, 5)
    
    # This process makes the background black
    image = image - np.mean(image)
    image = image / np.std(image)
    
    # Taking derivatives
    dx = ndimage.sobel(image, axis=0)
    dy = ndimage.sobel(image, axis=1)
    image = np.hypot(dx, dy)
    
    # For some reason we need to save the image to disk to get a nice picture
    scipy.misc.imsave('sobel.jpg', image)
    image = scipy.misc.imread("sobel.jpg")
    
    return image

In [5]:
class Labels(Enum):
    isai = 0
    pratt = 1
    david = 2
    jacob = 3
    helenita = 4
    classes = 5

In [6]:
def save_image(image):
    image.save("image.jpg")

In [7]:
def save_array_as_image(array, name, extension):
    im = Image.fromarray(np.uint8(array))
    im.save(name + "." + extension)

In [8]:
def crop(image):
    width, height, depth = image.shape   # Get dimensions
    new_width = 550
    new_height = 550

    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2
    
    lx, ly, lz = image.shape
    image = image[left:right, top:bottom]
    return image

In [9]:
def make_label_vector(file_path):
    file_name = ntpath.basename(file_path)
    underscore_index = file_name.index('_') + 1
    point_index = file_name.index('.')
    label_name = file_name[underscore_index:point_index]
    label_index = Labels[label_name].value
    
    label_vector = np.zeros((1,Labels.classes.value))
    label_vector[0,label_index] = 1 
    return label_vector

In [10]:
def randomly_process_image(image):
    random_number = uniform(-20,20)
    
    # This order works
    image = gradient_of_image(image)
    image = ndimage.rotate(image, random_number) 
    image = crop(image)
    image = scipy.misc.imresize(image, (HEIGHT, WIDTH, DEPTH))
    
    # Showing image for testing purposes
    #plt.imshow(image)
    #plt.show()
    return image

In [11]:
def make_features_vector(processed_image):
    features_vector = processed_image.reshape((1,HEIGHT*WIDTH*DEPTH))
    return features_vector

In [12]:
IMAGE_GENERATION = 10

def images_to_features_and_labels(folder, extension):

    features_matrix = np.zeros((0,HEIGHT * WIDTH * DEPTH))
    labels_matrix = np.zeros((0,Labels.classes.value))
    
    for file_path in glob.glob(folder + "/*." + extension):
        print(file_path)
        image = scipy.misc.imread(file_path)
        
        for i in range(IMAGE_GENERATION):
            label_vector = make_label_vector(file_path)
            processed_image = randomly_process_image(image)
            #features_vector = make_features_vector(processed_image)
            #features_matrix = np.vstack([features_matrix, features_vector])
            #labels_matrix = np.vstack([labels_matrix, label_vector])
            
    return features_matrix, labels_matrix

In [13]:
def save_matrix_to_disk(folder, name, matrix):
    np.save(folder + "/" + name, matrix)

In [14]:
def load_matrix_from_disk(folder, name):
    matrix = np.load(folder + "/" + name + ".npy")
    return matrix

In [15]:
def features_matrix_to_images(features_matrix):
    name_base = "image"
    extension = "jpg"
    for i in range(features_matrix.shape[0]):
        name = name_base + str(i)
        row = features_matrix[i]
        row = row.reshape((HEIGHT, WIDTH, DEPTH))
        save_array_as_image(row, name, extension)

In [16]:
features_matrix, labels_matrix = images_to_features_and_labels("./Images/Faces", "jpg")
save_matrix_to_disk("./", "features", features_matrix)
save_matrix_to_disk("./", "labels", labels_matrix)

./Images/Faces/2016-09-22-205950 (1)_isai.jpg
./Images/Faces/2016-09-22-205726 (1)_david.jpg
./Images/Faces/2016-09-22-205821_jacob.jpg
./Images/Faces/2016-09-22-205952 (1)_isai.jpg
./Images/Faces/2016-09-21-213026_isai.jpg
./Images/Faces/2016-09-22-123743 (1)_isai.jpg
./Images/Faces/2016-09-22-205926 (1)_helenita.jpg
./Images/Faces/2016-09-22-205841_jacob.jpg
./Images/Faces/2016-09-22-205851 (1)_helenita.jpg
./Images/Faces/2016-09-22-205927 (1)_helenita.jpg
./Images/Faces/2016-09-22-205851_helenita.jpg
./Images/Faces/2016-09-22-205952_isai.jpg
./Images/Faces/2016-09-22-205746 (1)_david.jpg
./Images/Faces/2016-09-21-213020_isai.jpg
./Images/Faces/2016-09-22-205740_david.jpg
./Images/Faces/2016-09-22-205956_isai.jpg
./Images/Faces/2016-09-22-205927 (2)_helenita.jpg
./Images/Faces/2016-09-22-123735 (1)_pratt.jpg
./Images/Faces/2016-09-22-205741_david.jpg
./Images/Faces/2016-09-22-205831_jacob.jpg
./Images/Faces/2016-09-22-205756_david.jpg
./Images/Faces/2016-09-22-205950_isai.jpg
./Image